Importación de librerías base para el problema 

In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import plotly.express as px
import os

os.getcwd()


'c:\\Users\\shado\\OneDrive\\Documentos\\NTT DATA\\LATAM'

Lectura de datos principal y creacion de copia auxiliar de de la data original, también se busco información extra sobre países de los destinos de vuelos y los días feriados de ese año para el origen de los vuelos

In [3]:
datos = pd.read_csv("dataset_SCL.csv", sep=",")
datos_original = datos.copy()
paises =  pd.read_excel("Prefijo Paises.xlsx", sheet_name= 'Hoja2')
feriados =  pd.read_excel("Feriados.xlsx")
datos.columns


C:\Users\shado\AppData\Local\Temp\ipykernel_7096\200450372.py:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  datos = pd.read_csv("dataset_SCL.csv", sep=",")


Index(['Fecha-I', 'Vlo-I', 'Ori-I', 'Des-I', 'Emp-I', 'Fecha-O', 'Vlo-O',
       'Ori-O', 'Des-O', 'Emp-O', 'DIA', 'MES', 'AÑO', 'DIANOM', 'TIPOVUELO',
       'OPERA', 'SIGLAORI', 'SIGLADES'],
      dtype='object')

verificación de los tipos de datos de este dataframe

In [4]:
datos.dtypes

Fecha-I      object
Vlo-I        object
Ori-I        object
Des-I        object
Emp-I        object
Fecha-O      object
Vlo-O        object
Ori-O        object
Des-O        object
Emp-O        object
DIA           int64
MES           int64
AÑO           int64
DIANOM       object
TIPOVUELO    object
OPERA        object
SIGLAORI     object
SIGLADES     object
dtype: object

transformanción de Fechas a formato tiempo

In [5]:
datos["Fecha-I"] = pd.to_datetime(datos["Fecha-I"])
datos["Fecha-O"] = pd.to_datetime(datos["Fecha-O"])
datos.dtypes

Fecha-I      datetime64[ns]
Vlo-I                object
Ori-I                object
Des-I                object
Emp-I                object
Fecha-O      datetime64[ns]
Vlo-O                object
Ori-O                object
Des-O                object
Emp-O                object
DIA                   int64
MES                   int64
AÑO                   int64
DIANOM               object
TIPOVUELO            object
OPERA                object
SIGLAORI             object
SIGLADES             object
dtype: object

creación de variables dif_min, hora_dia, hora, minutos

In [6]:
datos['dif_min'] = (datos["Fecha-O"]- datos["Fecha-I"])
datos['hora_dia'] = datos['Fecha-I'].dt.time
datos['hora'] = datos['Fecha-I'].dt.hour
datos['minutos'] = datos['Fecha-I'].dt.minute

datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,2017,Martes,I,American Airlines,Santiago,Miami,0 days 00:09:00,23:30:00,23,30
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,2017,Miercoles,I,American Airlines,Santiago,Miami,0 days 00:03:00,23:30:00,23,30
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,2017,Jueves,I,American Airlines,Santiago,Miami,-1 days +23:58:00,23:30:00,23,30


cambia la diferencia de las fechas de programación y salida de los vuelos a minutos.

In [7]:
datos["dif_min"] =  datos["dif_min"]/np.timedelta64(1,'m')

Homologación de variable Mes a nombre

In [8]:
conditions = [
    (datos['MES'] == 1),
    (datos['MES'] == 2),
    (datos['MES'] == 3),
    (datos['MES'] == 4),
    (datos['MES'] == 5),
    (datos['MES'] == 6),
    (datos['MES'] == 7),
    (datos['MES'] == 8),
    (datos['MES'] == 9),
    (datos['MES'] == 10),
    (datos['MES'] == 11),
    (datos['MES'] == 12)]
values = [
    'Enero',
    'Febrero',
    'Marzo',
    'Abril',
    'Mayo',
    'Junio',
    'Julio',
    'Agosto',
    'Septiembre',
    'Octubre',
    'Noviembre',
    'Diciembre']
datos['Mes_Nombre'] = np.select(conditions, values)
datos.head()

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,hora_dia,hora,minutos,Mes_Nombre
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,Domingo,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,Lunes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,...,Martes,I,American Airlines,Santiago,Miami,9.0,23:30:00,23,30,Enero
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,...,Miercoles,I,American Airlines,Santiago,Miami,3.0,23:30:00,23,30,Enero
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,...,Jueves,I,American Airlines,Santiago,Miami,-2.0,23:30:00,23,30,Enero


Verificacion que el valores este correcto

In [9]:
datos[["dif_min","Fecha-O","Fecha-I"]]

,dif_min,Fecha-O,Fecha-I
0,3.0,2017-01-01 23:33:00,2017-01-01 23:30:00
1,9.0,2017-01-02 23:39:00,2017-01-02 23:30:00
2,9.0,2017-01-03 23:39:00,2017-01-03 23:30:00
3,3.0,2017-01-04 23:33:00,2017-01-04 23:30:00
4,-2.0,2017-01-05 23:28:00,2017-01-05 23:30:00
...,...,...,...
68201,46.0,2017-12-22 15:41:00,2017-12-22 14:55:00
68202,16.0,2017-12-25 15:11:00,2017-12-25 14:55:00
68203,40.0,2017-12-27 15:35:00,2017-12-27 14:55:00
68204,13.0,2017-12-29 15:08:00,2017-12-29 14:55:00


Creacion de variable temporada_alta y atraso_15


In [10]:
datos['temporada_alta'] = np.where(  ((datos['Fecha-I']>='2017-12-15') & (datos['Fecha-I']<='2018-03-03')) 
                                   | ((datos['Fecha-I']>='2017-07-15') & (datos['Fecha-I']<='2017-07-31'))
                                   | ((datos['Fecha-I']>='2017-01-01') & (datos['Fecha-I']<='2017-03-03'))
                                   | ((datos['Fecha-I']>='2017-09-11') & (datos['Fecha-I']<='2017-09-30'))
                                   ,1,0)
datos['atraso_15'] = np.where(datos['dif_min']>15,1,0)




creacion de variable periodo_dia

In [11]:
conditions = [
    (datos['hora'] >= 5) & (datos['hora'] < 12),
    (datos['hora'] >= 12) & (datos['hora'] < 19),
    (datos['hora'] >= 19) | ((datos['hora'] >= 0) & (datos['hora'] < 5))]
values = ['mañana', 'tarde', 'noche']
datos['periodo_dia'] = np.select(conditions, values)


exportacion de archivo csv con las variables creadas desde el dataframe

In [12]:
synthetic_features = datos[['temporada_alta','dif_min','atraso_15','periodo_dia']]

synthetic_features.to_csv("synthetic_features.csv", sep="|"
                          , header=True, index=False)